In [32]:
import numpy as np, pandas as pd
import math, re, datetime, random
import pylab as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [682]:
def rescaling(x, d1, d2):
    if (x <= d1):
        return 'small'
    elif (x <= d2):
        return 'mid'
    else:
        return 'big'
     
def get_data3(ratio = 0.8):
    data = pd.read_csv('./data3.csv')
    data['好瓜'] = data['好瓜'].apply(lambda x: True if x == '是' else False)
    data = data.drop(axis=1,columns=['编号'])
    feature_rescale = ['含糖率', '密度']
    for fea in feature_rescale:
        sorted_feature_data = sorted(list(data[fea]))
        n = len(sorted_feature_data)
        data[fea] = data[fea].apply(rescaling,d1=sorted_feature_data[n//3],d2=sorted_feature_data[2*n//3])
    
    return data

In [688]:
class Naive_Bayes():
    def __init__(self, data_train):
        from collections import defaultdict
        self.data_train = data_train
        self.c0 = len(data_train[data_train[data_train.columns[-1]] == False])
        self.c1 = len(data_train[data_train[data_train.columns[-1]] == True])
        self.table = ({}, {})
        
    def fit(self):
        cols = self.data_train.columns
        self.table[0][cols[-1]] = (self.c0 + 1) / (self.c0 + self.c1 + 2)
        self.table[1][cols[-1]] = (self.c1 + 1) / (self.c0 + self.c1 + 2)
        
        for fea in cols[:-1]:
            fea_dic = [{}, {}]
            groups = self.data_train.groupby(axis = 0, by = fea)
            N = len(groups)
            
            fea_dic[0] = defaultdict(lambda: 1 / float(self.c0 + N))
            fea_dic[1] = defaultdict(lambda: 1 / float(self.c1 + N))
            
            for (val, tab) in groups:
                n0, n1 = len(tab[tab[tab.columns[-1]] == False]), len(tab[tab[tab.columns[-1]] == True])
                fea_dic[0][val] = (n0 + 1) / float(self.c0 + N)
                fea_dic[1][val] = (n1 + 1) / float(self.c1 + N)
            
            (self.table[0][fea], self.table[1][fea]) = fea_dic
    
    def test(self, sample):
        cols = self.data_train.columns
        logl = np.zeros(2)
        for c in range(2):
            logl[c] += np.log(self.table[c][cols[-1]])
            for fea in cols[:-1]:
                val = sample[fea]
                logl[c] += np.log(self.table[c][fea][val])
        output = [sample[-1], logl[1] >= logl[0], (logl[0], logl[1])]
        print ('{}, {} <-- {} '.format(output[0], output[1], output[2]))
        
        return output[0] == output[1]

In [689]:
class AODE_Bayes():
    def __init__(self, data_train):
        from collections import defaultdict
        self.data_train = data_train
        self.c = (len(data_train[data_train[data_train.columns[-1]]==False]), \
                  len(data_train[data_train[data_train.columns[-1]]==True]))
        self.ctable = ({}, {})
        self.table = ({}, {})
        
    def fit(self):
        cols = self.data_train.columns
        
        for C in range(2):
            
            data = self.data_train[self.data_train[cols[-1]] == (True if C == 1 else False)]
            
            for par in cols[:-1]:
                groups_par = data.groupby(axis = 0, by = par)
                N = len(groups_par)
                par_dic = defaultdict(lambda: 1 / float(self.c[0] + self.c[1] + 2 * N))
                self.table[C][par] = {}
                
                for (val, data_par) in groups_par:
                    NC = len(data_par)
                    par_dic[val] = (NC + 1) / float(self.c[0] + self.c[1] + 2 * N)
                    self.table[C][par][val] = {}
                    
                    for son in cols[:-1]:
                        if (son == par): 
                            continue
                        else:
                            groups_son = data_par.groupby(axis = 0, by = son)
                            n = len(groups_son)
                            son_dic = defaultdict(lambda: 1 / float(NC + n))
                            for (ual, data_son) in groups_son:
                                nC = len(data_son)
                                son_dic[ual] = (nC + 1) / (NC + n)
                        self.table[C][par][val][son] = son_dic
            
                self.ctable[C][par] = par_dic
            
    def test(self, sample):
        cols = data_train.columns
        logl = np.zeros(2)
        for C in range(2):
            for par in cols[:-1]:
                val = sample[par]
                if par not in self.table[C] or val not in self.table[C][par]: continue    
                p = self.ctable[C][par][val]
                for son in cols[:-1]:
                    if son == par: continue
                    ual = sample[son]
                    if son not in self.table[C][par][val]: continue
#                         print('par, val, son, ual = ({},{},{},{})'.format(par,val,son,ual))
                    p *= self.table[C][par][val][son][ual]
                logl[C] += p
        
        output = [sample[-1], logl[1] >= logl[0], (logl[0], logl[1])]
        print ('{}, {} <-- {} '.format(output[0], output[1], output[2]))
        
        return output[0] == output[1]

In [690]:
data_train = get_data3()
nb = Naive_Bayes(data_train)
nb.fit()
snb = AODE_Bayes(data_train)
snb.fit()

acc1, acc2, n = 0.0, 0.0, len(data_train)
for i in range(n):
    sample_test = data_train.iloc[i]
    acc1 += nb.test(sample_test)
    acc2 += snb.test(sample_test)

print ('test_accracy 1: {}'.format(acc1 / float(n)))
print ('test_accracy 2: {}'.format(acc2 / float(n)))

True, True <-- (-9.424027696633352, -6.409084615258397) 
True, True <-- (0.009127126293838538, 0.1039831390653319) 
True, True <-- (-9.934853320399345, -7.033238924331391) 
True, True <-- (0.013046840658126965, 0.12406379614193605) 
True, True <-- (-9.999391841536916, -5.451971888863987) 
True, True <-- (0.015923203906609656, 0.06963178372431475) 
True, True <-- (-9.934853320399345, -6.74555685187961) 
True, True <-- (0.01063625971650663, 0.10324877126949683) 
True, True <-- (-9.488566217770924, -6.368262620738142) 
True, True <-- (0.015849590741226362, 0.12746183156834975) 
True, True <-- (-9.249674309488576, -7.844169140547719) 
True, True <-- (0.009172010876290661, 0.06662907900865361) 
True, False <-- (-8.84420920138041, -9.412785058461566) 
True, True <-- (0.009774676400600662, 0.20095530684057483) 
True, True <-- (-8.977740594004933, -6.773727728846306) 
True, True <-- (0.012606073398720018, 0.06741426172807047) 
False, False <-- (-8.220054892307417, -9.923610682227556) 
False, F